In [2]:
import tensorflow as tf
from tensorflow import keras

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

print(tf.__version__)

ImportError: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy c-extensions failed.
- Try uninstalling and reinstalling numpy.
- If you have already done that, then:
  1. Check that you expected to use Python3.7 from "C:\Users\kids\anaconda3\python.exe",
     and that you have no directories in your PATH or PYTHONPATH that can
     interfere with the Python and numpy version "1.19.1" you're trying to use.
  2. If (1) looks fine, you can open a new issue at
     https://github.com/numpy/numpy/issues.  Please include details on:
     - how you installed Python
     - how you installed numpy
     - your operating system
     - whether or not you have multiple versions of Python installed
     - if you built from source, your compiler versions and ideally a build log

- If you're working with a numpy git repository, try `git clean -xdf`
  (removes all files not under version control) and rebuild numpy.

Note: this error has many possible causes, so p

In [ ]:
#Load and Scale the Data

path = "datasets/"
spy_df = pd.read_csv(path + 'spy_30min_pi_clean.csv')

spy_df["Date"] = pd.to_datetime(spy_df["Date"])

#Created scaled sp500 dataframe.
scaled_spy_df = spy_df.copy()

#Pivot the data: vertical axis is date, horizontal axis is time of day.
scaled_spy_df = scaled_spy_df.pivot(index = "Time", columns = "Date", values = "Close")

scaled_spy_df

prev_close=0
count=0

for col in scaled_spy_df.columns:
    last_close=0
    start_close = scaled_spy_df[col].iloc[0]
    close=scaled_spy_df[col][-1]
    min_close = scaled_spy_df[col].min()
    max_close = scaled_spy_df[col].max()
    
    if (count==0):
        last_close= scaled_spy_df[col][0]
    else:
        last_close=prev_close
    prev_close= scaled_spy_df[col][-1]
    
    scaled_spy_df[col] = scaled_spy_df[col].apply(lambda x : (x-last_close)/last_close)
    count+=1

scaled_spy_df

In [ ]:
#Preprocess the Data

#Convert DataFrame into a Numpy array.
spy_np = scaled_spy_df.to_numpy()
spy_np = spy_np.T

#No scale:
scaled_spy_np = spy_np
#Scale:
#print("SPY STD:", spy_np.std())
#scaled_spy_np = spy_np/spy_np.std()

#Split data into input and output.
X = scaled_spy_np[:,0:8] #input
y = scaled_spy_np[:,8:13] #what we're predicting

print("scaled_spy_np has length {}".format(len(scaled_spy_np[0])))
print("X has length {} \ny has length {}".format(len(X[0]), len(y[0])))

#Split data into train and test.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .1)

In [ ]:
print(len(X_train))
print(len(X_train[0]))

In [ ]:
#Create the model.
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(units=128, activation='tanh'))
model.add(tf.keras.layers.Dense(units=5, activation='softmax'))

#Compile the model.
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

#Train the model.
model.fit(X_train, y_train, epochs=5)

#Evaluate the model.
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test accuracy: {}".format(test_accuracy))